In [18]:
import spacy
import pdfplumber

In [12]:
!python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.7 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
# Load the "en_core_web_sm" pipeline
nlp = spacy.load("en_core_web_sm")

In [25]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

def categorize_resume(text):
    doc = nlp(text)

    categorized_data = {
        "education": "",
        "skills": "",
        "experience": ""
    }

    capture_mode = None

    for sent in doc.sents:
        if "PROFESSIONAL EXPERIENCE" in sent.text:
            capture_mode = "experience"
            continue
        elif "EDUCATION" in sent.text:
            capture_mode = "education"
            continue
        elif "SKILLS & TOOLS" in sent.text or "COMPETENCIES" in sent.text:
            capture_mode = "skills"
            continue

        if capture_mode:
            if any(section in sent.text for section in ["PROFESSIONAL EXPERIENCE", "EDUCATION", "SKILLS & TOOLS", "COMPETENCIES"]):
                capture_mode = None
            else:
                categorized_data[capture_mode] += sent.text + "\n"

    return categorized_data






In [28]:
def categorize_resume_2(text):
    doc = nlp(text)
    categorized_data = {
        "education": "",
        "skills": "",
        "experience": ""
    }

    lines = text.split('\n')

    # Extracting experiences
    experience_index = lines.index("PROFESSIONAL EXPERIENCE")
    for line in lines[experience_index+1:]:
        if line in ["EDUCATION", "SKILLS & TOOLS", "COMPETENCIES"]:
            break
        categorized_data["experience"] += line + "\n"
    
    # Extracting education
    education_index = lines.index("EDUCATION")
    for line in lines[education_index+1:]:
        if line in ["SKILLS & TOOLS", "COMPETENCIES", "PROFESSIONAL EXPERIENCE"]:
            break
        categorized_data["education"] += line + "\n"
    
    # Extracting skills
    if "SKILLS & TOOLS" in lines:
        skills_index = lines.index("SKILLS & TOOLS")
        for line in lines[skills_index+1:]:
            if line in ["EDUCATION", "COMPETENCIES", "PROFESSIONAL EXPERIENCE"]:
                break
            categorized_data["skills"] += line + "\n"
    
    return categorized_data

In [29]:
if __name__ == "__main__":
    # readResume("/Users/mac/Eti/CompetentPro/Competent_Profiles/src/components/pdf/a.pdf")
    
    pdf_path = "/Users/mac/Eti/CompetentPro/Competent_Profiles/src/components/pdf/a.pdf"
    resume_text = extract_text_from_pdf(pdf_path)

    categorized_data = categorize_resume_2(resume_text)
    for category, items in categorized_data.items():
        print(category, items)


education ITMO University, Russia Sep 2022 - Jul 2024
Master of Science - MS, Big Data & Machine Learning
Harvard University, Remote 2021
Diploma in Business Analytics, Financial Accounting, & Economics for Managers
Bangalore University, India 2010 - 2013
Bachelors of Computer Applications

skills Skills: Python programming, SQL querying, Real-time video data analysis, Machine learning
algorithms, Web development, Project management, Cross-functional team collaboration,
Creative problem-solving, Software architecture design and implementation, Recruiting
process optimization, Technical documentation, API integration, Data modeling and
database design, Agile development methodologies Version control systems (e.g. Git),
Hugging Face, Big Data, PyTorch, Tensorflow, Keras

experience Geek Radius, Lagos, Nigeria Jan 2020 - Present
Founder
● Founded and led Geek Radius in Jan 2020, a pioneering creative-tech agency
providing an all-encompassing environment for creatives to thrive in the digi